In [1]:
import numpy as np
import keras
from keras.layers import Dense, Input, Add
from keras.models import Sequential, Model, load_model
import scipy.special as sp

from Source import AWGN

Using TensorFlow backend.


In [2]:
n_power=7
CodeRate=0.5
N_length=int(2**n_power)
msg_length=int(CodeRate*N_length)
frozen_length=N_length-msg_length
codeNum =10**6
test_codeNum = 10**3

EbNo_Start=1.0  # in dB
EbNo_delta=0.5 # in dB
EbNo_Numbers=10 
EbNo=np.arange(EbNo_Start,EbNo_Start+EbNo_Numbers*EbNo_delta,EbNo_delta)
ebno=10.**(EbNo/10.)          # dB to real value 변환
BER_BPSK_Uncoded=0.5*sp.erfc(np.sqrt(ebno))      # BPSK BER 공식

In [3]:
G_All=AWGN.AllGeneratorMatrixPolarCode(n_power) ###생성행렬생성

In [4]:
(Z_AWGN,Z_decending_index_AWGN)=AWGN.BhanttacharyyaParameterAWGN(N_length, n_power, CodeRate)

In [5]:
SelectedBitsPositions = Z_decending_index_AWGN[frozen_length:]
SelectedBitsPositions.sort()
FrozenBitsPositions = Z_decending_index_AWGN[:frozen_length]
FrozenBitsPositions.sort()

In [6]:
msg=np.random.randint(0,2,size=(codeNum, msg_length))

In [7]:
msg,codeword, uncodedbits = AWGN.Encoding(msg, codeNum, N_length, n_power, msg_length, SelectedBitsPositions, G_All[n_power])
ReceivedAWGN = AWGN.ReceivedCode(ebno[0], CodeRate,codeNum, N_length, codeword)


In [8]:
# test_msg=np.random.randint(0,2,size=(test_codeNum, msg_length))

In [9]:
# test_msg, test_codeword, test_uncodedbits = AWGN.Encoding(test_msg, test_codeNum, N_length, n_power, msg_length, SelectedBitsPositions, 
#                                         G_All[n_power]) 
test_ReceivedAWGN = AWGN.ReceivedCode(ebno[0], CodeRate,codeNum, N_length, codeword)


In [10]:
input_layer = Input(shape=(128,))

line1_Dense1 = Dense(128*1, activation='elu')(input_layer)
line1_Dense2 = Dense(128*1, activation='elu')(line1_Dense1)         
line1_Dense3 = Dense(128*1, activation='elu')(line1_Dense2)
line1_output_layer = Dense(128, activation='tanh')(line1_Dense3)

line2_Dense1 = Dense(128*2, activation='elu')(input_layer)
line2_Dense2 = Dense(128*4, activation='elu')(line2_Dense1)         
line2_Dense3 = Dense(128*2, activation='elu')(line2_Dense2)
line2_output_layer = Dense(128, activation='tanh')(line2_Dense3)

line3_Dense1 = Dense(128*3, activation='elu')(input_layer)
line3_Dense2 = Dense(128*9, activation='elu')(line3_Dense1)         
line3_Dense3 = Dense(128*3, activation='elu')(line3_Dense2)
line3_output_layer = Dense(128, activation='tanh')(line3_Dense3)

line4_Dense1 = Dense(128*4, activation='elu')(input_layer)
line4_Dense2 = Dense(128*16, activation='elu')(line4_Dense1)         
line4_Dense3 = Dense(128*4, activation='elu')(line4_Dense2)
line4_output_layer = Dense(128, activation='tanh')(line4_Dense3)

line5_Dense1 = Dense(128*5, activation='elu')(input_layer)
line5_Dense2 = Dense(128*25, activation='elu')(line5_Dense1)         
line5_Dense3 = Dense(128*5, activation='elu')(line5_Dense2)
line5_output_layer = Dense(128, activation='tanh')(line5_Dense3)

line6_Dense1 = Dense(128*6, activation='elu')(input_layer)
line6_Dense2 = Dense(128*36, activation='elu')(line6_Dense1)         
line6_Dense3 = Dense(128*6, activation='elu')(line6_Dense2)
line6_output_layer = Dense(128, activation='tanh')(line6_Dense3)

add_layer = Add()([line1_output_layer, line2_output_layer, line3_output_layer, line4_output_layer, 
                   line5_output_layer, line6_output_layer])


output_layer = Dense(64, activation='sigmoid')(add_layer)

model = Model(inputs = input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
model.fit(ReceivedAWGN, msg, 
          epochs=25,
          batch_size=10000,
          shuffle=True,
          validation_data=(test_ReceivedAWGN,msg))


Train on 1000000 samples, validate on 1000000 samples
Epoch 1/25
1000000/1000000 [==============================] - 23s 23us/step - loss: 0.7168 - accuracy: 0.5215 - val_loss: 0.6733 - val_accuracy: 0.5360
Epoch 2/25
1000000/1000000 [==============================] - 22s 22us/step - loss: 0.6695 - accuracy: 0.5392 - val_loss: 0.6680 - val_accuracy: 0.5383
Epoch 3/25
1000000/1000000 [==============================] - 22s 22us/step - loss: 0.6657 - accuracy: 0.5441 - val_loss: 0.6662 - val_accuracy: 0.5407
Epoch 4/25
1000000/1000000 [==============================] - 22s 22us/step - loss: 0.6638 - accuracy: 0.5486 - val_loss: 0.6647 - val_accuracy: 0.5439
Epoch 5/25
1000000/1000000 [==============================] - 22s 22us/step - loss: 0.6623 - accuracy: 0.5533 - val_loss: 0.6639 - val_accuracy: 0.5475
Epoch 6/25
1000000/1000000 [==============================] - 22s 22us/step - loss: 0.6607 - accuracy: 0.5587 - val_loss: 0.6620 - val_accuracy: 0.5534
Epoch 7/25
1000000/1000000 [=====

In [ ]:
loss = model.evaluate(test_ReceivedAWGN, msg, verbose=0)
print(loss)

In [13]:
model.save('Model/model_AWGN_p6_2.h5')